In [1]:
import sys

# Load diabetes dataset from sklearn
from sklearn.datasets import load_iris

import datagnosis.logger as log
log.add(sink=sys.stderr, level="INFO")

In [2]:
X, y = load_iris(return_X_y=True, as_frame=True)
df = X.copy(deep=True)
df['target'] = y
display(df)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:

from datagnosis.plugins.core.datahandler import DataHandler
from datagnosis.plugins.core.models.simple_mlp import SimpleMLP
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

std_scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

datahander = DataHandler(X_train, y_train, batch_size=32)


model = SimpleMLP()
# creating our optimizer and loss function object
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [4]:
# datagnosis absolute
from datagnosis.plugins import Plugins

plugins = Plugins().list()
print(plugins)


['prototypicality', 'confident_learning', 'grand', 'large_loss', 'data_iq', 'aum', 'el2n', 'vog', 'allsh', 'conf_agree', 'forgetting', 'data_maps']


In [5]:
hcm = Plugins().get(
    "vog",
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    lr=learning_rate,
    epochs=10,
    num_classes=3,
    logging_interval=1,
)
hcm.fit(
    datahandler=datahander,
    use_caches_if_exist=True,
)

[2023-07-18T11:39:24.335237+0100][87263][INFO] Fitting allsh plugin
[2023-07-18T11:39:25.430528+0100][87263][INFO] Epoch 1/10: Loss=0.6950
[2023-07-18T11:39:25.437744+0100][87263][INFO] Epoch 2/10: Loss=0.2969
[2023-07-18T11:39:25.444881+0100][87263][INFO] Epoch 3/10: Loss=0.2084
[2023-07-18T11:39:25.451409+0100][87263][INFO] Epoch 4/10: Loss=0.1043
[2023-07-18T11:39:25.457786+0100][87263][INFO] Epoch 5/10: Loss=0.1239
[2023-07-18T11:39:25.464612+0100][87263][INFO] Epoch 6/10: Loss=0.0880
[2023-07-18T11:39:25.472771+0100][87263][INFO] Epoch 7/10: Loss=0.0534
[2023-07-18T11:39:25.480672+0100][87263][INFO] Epoch 8/10: Loss=0.0516
[2023-07-18T11:39:25.488869+0100][87263][INFO] Epoch 9/10: Loss=0.0451
[2023-07-18T11:39:25.495889+0100][87263][INFO] Epoch 10/10: Loss=0.0364
[2023-07-18T11:39:25.496460+0100][87263][INFO] updating allsh plugin


ValueError: pic should be 2/3 dimensional. Got 1 dimensions.

In [ ]:
hcm_scores = hcm.scores
# print(hcm_scores)
print(hcm.score_names)
hcm.plot_scores(axis=1, plot_type="scatter")

In [ ]:
import pandas as pd
print(hcm.name())
print(hcm.hard_direction())
hardest_5 = hcm.extract_datapoints(method="index", indices=[0,1,2,3,4,5])

display(pd.DataFrame(
    data={
        "indices":hardest_5[0][2],
        f"{X.columns[0]}": hardest_5[0][0].transpose(0,1)[0],
        f"{X.columns[1]}": hardest_5[0][0].transpose(0,1)[1],
        f"{X.columns[2]}": hardest_5[0][0].transpose(0,1)[2],
        f"{X.columns[3]}": hardest_5[0][0].transpose(0,1)[3],
        "labels": hardest_5[0][1],
        "scores": hardest_5[1],
    }
))